In [64]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Bidirectional

In [40]:
data=pd.read_csv('train_data.csv')
test=pd.read_csv('test_data.csv')
data.head()

,lable,text
0,1,ounc feather bowl hummingbird opec moment alab...
1,1,wulvob get medirc onlin qnb ikud viagra escape...
2,0,comput connect cnn com wednesday escapenumb ma...
3,1,univers degre obtain prosper futur money earn ...
4,0,thank answer guy know check rsync manual would...


In [47]:
data=data.dropna()

In [48]:
len(data)

62349

In [50]:
data.reset_index(inplace=True)

In [ ]:
m=0
for i in range(len(data)):
  print(len((data['text'][i]).split()))

In [53]:
len(data)

62349

In [55]:
one_hut=[one_hot(words,10000) for words in data['text']]

In [63]:
sent_length=500
embedded_docs=pad_sequences(one_hut,padding='pre',maxlen=sent_length)

embedded_docs

array([[   0,    0,    0, ..., 1265,  787, 8909],
       [   0,    0,    0, ..., 8001, 6108, 5652],
       [   0,    0,    0, ..., 3487, 3117, 2623],
       ...,
       [   0,    0,    0, ..., 2714, 3657, 9726],
       [   0,    0,    0, ..., 6241, 3365,  933],
       [   0,    0,    0, ..., 2714, 3657, 9726]], dtype=int32)

In [66]:
embedding_vector_features=500
model=Sequential()
model.add(Embedding(10000,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.build(input_shape=(None,sent_length))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 500, 500)            │       5,000,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 200)                 │         480,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             201 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,481,001 (20.91 MB)

 Trainable params: 5,481,001 (20.91 MB)

 Non-trainable params: 0 (0.00 B)

None


In [69]:
test=test.dropna()
test.reset_index(inplace=True)

In [70]:
one_hut=[one_hot(words,10000) for words in test['text']]

In [72]:
sent_length=500
embedded_docs_test=pad_sequences(one_hut,padding='pre',maxlen=sent_length)

embedded_docs_test

array([[   0,    0,    0, ..., 3494, 1800, 4099],
       [5224, 5652, 5338, ..., 5652, 5652, 5652],
       [   0,    0,    0, ..., 1838, 5789,  385],
       ...,
       [   0,    0,    0, ..., 2053, 1418, 4830],
       [   0,    0,    0, ..., 5652, 8496, 4000],
       [   0,    0,    0, ..., 8298, 9843, 8933]], dtype=int32)

In [74]:
X_train=np.array(embedded_docs)
y_train=np.array(data['lable'])
X_test=np.array(embedded_docs_test)
y_test=np.array(test['lable'])

In [75]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=20,batch_size=32)

Epoch 1/20
1949/1949 ━━━━━━━━━━━━━━━━━━━━ 123s 60ms/step - accuracy: 0.9383 - loss: 0.1627 - val_accuracy: 0.9820 - val_loss: 0.0611
Epoch 2/20
1949/1949 ━━━━━━━━━━━━━━━━━━━━ 139s 60ms/step - accuracy: 0.9866 - loss: 0.0395 - val_accuracy: 0.9846 - val_loss: 0.0462
Epoch 3/20
1949/1949 ━━━━━━━━━━━━━━━━━━━━ 142s 60ms/step - accuracy: 0.9941 - loss: 0.0180 - val_accuracy: 0.9839 - val_loss: 0.0524
Epoch 4/20
 154/1949 ━━━━━━━━━━━━━━━━━━━━ 1:35 53ms/step - accuracy: 0.9940 - loss: 0.0148

KeyboardInterrupt: 

In [76]:
y_pred=model.predict(X_test)

659/659 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step


In [77]:
y_pred=np.where(y_pred>=0.5,1,0)

In [78]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[ 9845,   148],
       [  179, 10896]])

In [79]:
print(accuracy_score(y_test,y_pred))

0.9844788304537687


In [83]:
pr=input()
oh=one_hot(pr,10000)
oh=pad_sequences([oh],padding='pre',maxlen=sent_length)
model.predict(oh)

hello, i am great how are you
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


array([[0.99866605]], dtype=float32)